In [4]:
import cv2
import numpy as np
print(f"OpenCV version is: {cv2.__version__}")

# In Colab we need to use:
from google.colab.patches import cv2_imshow

ModuleNotFoundError: No module named 'cv2.cv2'

In [ ]:
%cd /content/
!rm -rf rc-2023-24
!git clone https://github.com/mim-uw/rc-2023-24.git
%cd rc-2023-24/docs/lab5-public/data

In [ ]:
# Camera parameteres
camera_matrix = np.array([
    [1.28613634e+03, 0.00000000e+00, 6.92097797e+02],
    [0.00000000e+00, 1.30040995e+03, 4.25368745e+02],
    [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
])

dist_coeffs = np.array(
    [[-0.30566098, -0.17641842, -0.00495141, -0.00106297, 0.72164286]])

In [ ]:
# Calculate undistorted camera matrix

# OpenCV order
img = cv2.imread("hw11.jpg")
size = (img.shape[1], img.shape[0])

alpha = 0.
rect_camera_matrix = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, size, alpha)[0]

# Calculate undistortion maps
map1, map2 = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, np.eye(3), rect_camera_matrix, size, cv2.CV_32FC1)

undistorted_imgs = []

# Undistort the photos
for i in range(1, 5):
    img = cv2.imread("hw1" + str(i) + ".jpg")

    # Use maps to undistort an image
    rect_img = cv2.remap(img, map1, map2, cv2.INTER_LINEAR)

    # Show original and undistorted side by side
    cv2_imshow(cv2.hconcat([img, rect_img]))

    undistorted_imgs.append(rect_img)

In [ ]:
# Function for finding and displaying aruco markers
def find_aruco_markers(dictionary, img):
    detectorParams = cv2.aruco.DetectorParameters()
    detectorParams.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_CONTOUR
    detector = cv2.aruco.ArucoDetector(dictionary, detectorParams)

    img_draw = img.copy()
    corners, ids, _ = detector.detectMarkers(img)

    # inspect the image and draw detection
    RED = (0, 0, 255)
    cv2.aruco.drawDetectedMarkers(img_draw, corners, borderColor=RED)
    cv2_imshow(img_draw)

    return corners, ids

In [ ]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
corners_ls = []
ids_ls = []
for img in undistorted_imgs:
    corners, ids = find_aruco_markers(dictionary, img)
    corners_ls.append(corners)
    ids_ls.append(ids)

In [ ]:
subset_of_imgs = np.array([0, 1, 2]) # define the subset of size 3, for instance [0, 2, 3]
corners_ls = np.array(corners_ls)[subset_of_imgs]
ids_ls = np.array(ids_ls)[subset_of_imgs]

# TODO find the correct sequence of imgs

In [ ]:
def find_nearest_neighbor(p, shape):
    pos_x = np.array([np.floor(p[0]), np.ceil(p[0])])
    pos_y = np.array([np.floor(p[1]), np.ceil(p[1])])
    # find the nearest neighbor
    min_dis = np.inf
    x_min, y_min = 0, 0
    for x in pos_x:
        for y in pos_y:
            dis, x_aux, y_aux = np.inf, 0, 0
            if 0 <= x < shape[0] and 0 <= y < shape[1]:
                dis = np.sqrt((p[0] - x) ** 2 + (p[1] - y) ** 2)
                x_aux, y_aux = x, y
            elif 0 <= x < shape[0]:
                if y >= shape[1]:
                    dis = np.abs(p[1] - shape[1] + 1)
                    x_aux, y_aux = x, shape[1] - 1
                else:
                    dis = np.abs(p[1])
                    x_aux, y_aux = x, 0
            elif 0 <= y < shape[1]:
                if x >= shape[0]:
                    dis = np.abs(p[0] - shape[0] + 1)
                    x_aux, y_aux = shape[0] - 1, y
                else:
                    dis = np.abs(p[0])
                    x_aux, y_aux = 0, y
            else:
                corners_x = [0, shape[0] - 1]
                corers_y = [0, shape[1] - 1]
                for c_x in corners_x:
                    for c_y in corers_y:
                        dis_aux = np.sqrt((p[0] - c_x) ** 2 + (p[1] - c_y) ** 2)
                        if dis_aux < dis:
                            dis = dis_aux
                            x_aux, y_aux = c_x, c_y
            if dis < min_dis:
                min_dis = dis
                x_min, y_min = x_aux, y_aux

    return int(x_min), int(y_min)

def proj_trans(img, trans_mat, width):
    cv2_imshow(img)
    inv_trans_mat =  np.linalg.inv(trans_mat)
    # create a grid of points
    x, y = np.meshgrid(np.arange(img.shape[0]), np.arange(width))
    # create a matrix of points
    points = np.vstack((x.flatten(), y.flatten(), np.ones(img.shape[0] * width)))
    # apply the homography transformation
    points = inv_trans_mat @ points
    points = points / points[2, :]
    n_x, n_y = [], []
    for p in points.T:
        p_x, p_y = find_nearest_neighbor(p, img.shape)
        n_x.append(p_x)
        n_y.append(p_y)
    img_transformed = np.zeros((img.shape[0], width, 3), dtype=np.uint8)
    img_transformed[x.flatten(), y.flatten()] = img[n_x, n_y]
    cv2_imshow(img_transformed)

    return img_transformed

In [ ]:
proj_trans(img, np.array([[1., 0., 1.], [2., 1., 1.], [0., 0., 1.]]), 2 * img.shape[1])

In [ ]:
# Find homography matrix based on shared points using linalg.svd
def find_homography(shared_points):
    A = np.zeros((2 * len(shared_points), 9))
    for i in range(len(shared_points)):
        x1, y1 = shared_points[i][0][0], shared_points[i][0][1]
        x2, y2 = shared_points[i][1][0], shared_points[i][1][1]
        A[2 * i] = np.array([x1, y1, 1, 0, 0, 0, -x1 * x2, -x2 * y1, -x2])
        A[2 * i + 1] = np.array([0, 0, 0, x1, y1, 1, -x1 * y2, -y2 * y1, -y2])
    _, _, V = np.linalg.svd(A)

    return V[-1].reshape(3, 3)

In [ ]:
def random_tests_for_find_homography(number_of_tests):
    for t_n in range(number_of_tests):
        H = np.random.randn(3, 3)
        first_img_points = np.random.randn(2, 5)
        first_img_points = np.vstack((first_img_points, np.ones(5)))
        second_img_points = H @ first_img_points
        second_img_points = second_img_points / second_img_points[2]
        aux_shared_points = np.vstack((first_img_points[:2], second_img_points[:2]))
        shared_points = []
        for i in range(5):
            shared_points.append(aux_shared_points[:, i].reshape(2, 2))
        shared_points = np.array(shared_points)
        found_h = find_homography(shared_points)
        scale = H / found_h # the homography is the same up to a scaling factor
        assert(np.allclose(np.ones((3, 3)), scale / scale[0, 0]))
        print(f"TEST {t_n}: OK")

In [ ]:
# Run tests for the homography implementation
random_tests_for_find_homography(number_of_tests=100)

In [ ]:
# points shared between hw11.jpg and hw12.jpg for TASK 4
shared_points = np.array([[[466, 131], [449, 562]], [[472, 131], [455, 562]], [[466, 137], [449, 568]], [[472, 137], [455, 568]], [[472, 133], [455, 564]]])
hom = find_homography(shared_points)
hw12_trans = proj_trans(undistorted_imgs[1], hom, 1960)

In [ ]:
# stitch two images as a panorama where the overlapping region is the average of the two images
def panorama_without_weighted_average(img1, img2):
    stitched_img = np.zeros((img1.shape[0], img2.shape[1], 3), dtype=np.uint8)
    stitched_img[:, :img1.shape[1], :] = img1
    stitched_img += img2
    stitched_img[:, :img1.shape[1], :] //= 2

    return stitched_img

In [ ]:
cv2_imshow(panorama_without_weighted_average(undistorted_imgs[0], hw12_trans))

In [ ]:
# stitch two images as a panorama where the overlapping region is a weighted average of the two images
# depending on the distance from the center of the overlapping region
def panorama_with_weighted_average(img1, img2):
    stitched_img = np.zeros((img1.shape[0], img2.shape[1], 3), dtype=np.uint8)
    x_1, y_1 = np.meshgrid(np.arange(img1.shape[1]), np.arange(img1.shape[0]))
    x_2, y_2 = x_1, y_1
    x_1 -= img1.shape[0] // 2
    y_1 -= img1.shape[1] // 2
    x_1, y_1 = x_1 ** 2, y_1 ** 2
    x_2 -= img2.shape[0] // 2
    y_2 -= img2.shape[1] // 2
    x_2, y_2 = x_2 ** 2, y_2 ** 2
    z_1 = x_1 + y_1
    z_2 = x_2 + y_2
    z_1 = z_1 / (z_1 + z_2)
    z_2 = z_2 / (z_1 + z_2)
    for c in range(3):
        stitched_img[:, :img1.shape[1], c] = img1[:, :, c] * z_2 + img2[:, :img1.shape[1], c] * z_1

    stitched_img[:, img1.shape[1]:, :] = img2[:, img1.shape[1]:, :]

    return stitched_img

In [ ]:
cv2_imshow(panorama_with_weighted_average(undistorted_imgs[0], hw12_trans))